In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import csv


In [3]:
# Read data 

def readTrafficSigns(rootpath, target_size=(32, 32)):

    images = [] 
    labels = [] 

    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '05d') + '/' 
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') 
        gtReader = csv.reader(gtFile, delimiter=';') 
        next(gtReader)

        for row in gtReader:
            image = plt.imread(prefix + row[0])
            image = cv2.resize(image, target_size)
            images.append(image)
            labels.append(int(row[7]))
        gtFile.close()

    return images, labels

In [4]:
images, labels = readTrafficSigns('./Images')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [6]:
model = keras.Sequential([

    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(128, activation='relu'),
    
    layers.Dense(43, activation='softmax')  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [8]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
1931/1931 [==============================] - 21s 11ms/step - loss: 0.9341 - accuracy: 0.7926 - val_loss: 0.2485 - val_accuracy: 0.9314
Epoch 2/10
1931/1931 [==============================] - 21s 11ms/step - loss: 0.2272 - accuracy: 0.9422 - val_loss: 0.1504 - val_accuracy: 0.9617
Epoch 3/10
1931/1931 [==============================] - 21s 11ms/step - loss: 0.1710 - accuracy: 0.9587 - val_loss: 0.3395 - val_accuracy: 0.9490
Epoch 4/10
1931/1931 [==============================] - 22s 11ms/step - loss: 0.1356 - accuracy: 0.9678 - val_loss: 0.2141 - val_accuracy: 0.9605
Epoch 5/10
1931/1931 [==============================] - 22s 12ms/step - loss: 0.1221 - accuracy: 0.9725 - val_loss: 0.1659 - val_accuracy: 0.9661
Epoch 6/10
1931/1931 [==============================] - 22s 11ms/step - loss: 0.1372 - accuracy: 0.9718 - val_loss: 0.3159 - val_accuracy: 0.9369
Epoch 7/10
1931/1931 [==============================] - 22s 11ms/step - loss: 0.1226 - accuracy: 0.9759 - val_loss: 0.2433 -

In [9]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

242/242 [==============================] - 1s 4ms/step - loss: 0.1910 - accuracy: 0.9759
Test Accuracy: 0.9759129881858826
